In [61]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta
from jupyter_dash import JupyterDash
from dash import Dash
# Initialize the Dash app
app = JupyterDash(__name__)




# Load data
path = r"C:\Users\user\Downloads\Data Analyst Interview_sample_ecommerce_transactions (1).csv"
df = pd.read_csv(path)
df.head()



c:\Users\user\Desktop\data analysis project\venv\Lib\site-packages\dash\dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



,OrderID,OrderDate,CustomerID,CustomerName,Country,Product,Category,Quantity,UnitPrice,TotalPrice,PaymentMethod
0,ORD00001,2024-05-10,CUST2017,Customer_257,Canada,Sneakers,Clothing,4,1364.83,5459.32,PayPal
1,ORD00002,2024-05-15,CUST1042,Customer_44,Germany,Monitor,Electronics,9,924.6,8321.40,Credit Card
2,ORD00003,2023-05-30,CUST4149,Customer_54,UK,Jeans,Clothing,ten,none,11696.56,Bank Transfer
3,ORD00004,2023-07-23,CUST1882,Customer_80,Germany,T-shirt,Clothing,10,357.59,3575.90,Bank Transfer
4,ORD00005,32/13/2023,CUST9540,Customer_92,USA,Toaster,Home & Kitchen,8,739.19,5913.52,NaN


In [62]:
df["OrderDate"] =pd.to_datetime (df["OrderDate"], errors = "coerce")
df = df.dropna(subset=['OrderDate'])

# Group by date to get total quantity sold per day
daily_sales = df.groupby(df['OrderDate'].dt.date)['Quantity'].sum().reset_index()
daily_sales.columns = ['Date', 'TotalQuantity']
daily_sales['Date'] = pd.to_datetime(daily_sales['Date'])
print("Daily sales data created. Number of days:", len(daily_sales))
print("First few rows of daily_sales:\n", daily_sales.head())


# Sort by date
daily_sales = daily_sales.sort_values('Date')

# Initialize an empty DataFrame for live updates
live_data = pd.DataFrame({'Date': [], 'TotalQuantity': []})


Daily sales data created. Number of days: 308
First few rows of daily_sales:
         Date TotalQuantity
0 2023-01-04             2
1 2023-01-05             0
2 2023-01-07             3
3 2023-01-08             8
4 2023-01-09            15


In [63]:

# Layout of the dashboard
app.layout = html.Div([
    html.H1('Live Daily Sales Dashboard', style={'textAlign': 'center'}),
    dcc.Graph(id='sales-graph'),
    dcc.Interval(
        id='timer',
        interval=2*1000,  # Update every 2 seconds
        n_intervals=0
    ),
    dcc.Store(id='live-data-store'),  # Store live data
    html.Div(id='error-message', style={'color': 'red', 'textAlign': 'center'})
])

In [64]:
# Callback to update the graph
@app.callback(
    [Output('sales-graph', 'figure'), Output('live-data-store', 'data'), Output('error-message', 'children')],
    Input('timer', 'n_intervals'),
    Input('live-data-store', 'data')
)
def update_graph(n, stored_data):
    global daily_sales, live_data
    
    # Initialize live_data from stored data or empty
    if stored_data is not None:
        live_data = pd.DataFrame(stored_data)
        live_data['Date'] = pd.to_datetime(live_data['Date'])
    else:
        live_data = pd.DataFrame({'Date': [], 'TotalQuantity': []})
    
    # If there are still dates to add
    if n < len(daily_sales):
        # Add the next day's data
        new_row = daily_sales.iloc[n:n+1]
        live_data = pd.concat([live_data, new_row], ignore_index=True)
        print(f"Update {n}: Added data for {new_row['Date'].iloc[0].date()}. Live data rows: {len(live_data)}")
    
    # Check if live_data is empty
    if live_data.empty:
        fig = px.line(title='Total Items Sold Per Day')
        fig.add_annotation(text="No data available yet. Waiting for updates...", xref="paper", yref="paper", x=0.5, y=0.5, showarrow=False)
        error_message = "No data loaded. Check if clean.csv is correct."
    else:
        # Create line graph
        fig = px.line(
            live_data,
            x='Date',
            y='TotalQuantity',
            title='Total Items Sold Per Day'
        )
        error_message = ""
    
    # Update layout
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title="Total Items Sold",
        yaxis_range=[0, live_data['TotalQuantity'].max() * 1.2 if not live_data.empty else 100],
        xaxis_tickformat='%Y-%m-%d'
    )
    
    # Convert dates to string for JSON serialization
    live_data_store = live_data.copy()
    live_data_store['Date'] = live_data_store['Date'].astype(str)
    
    return fig, live_data_store.to_dict('records'), error_message



In [65]:
from dash import Dash, html

app = Dash(__name__)

app.layout = html.Div("Hello Dash!")

# This opens in the browser instead of trying to render inside the notebook
app.run(debug=True)


